<a href="https://colab.research.google.com/github/Mark4815/SQLviaColab/blob/main/SQLAnalysis_Covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#upload the data-set to Colab
from google.colab import files
uploaded = files.upload()

Saving COVID-19 Coronavirus.csv to COVID-19 Coronavirus.csv


In [ ]:
#import pandas library & duckdb library
import pandas as pd
import duckdb

In [ ]:
#create a dataframe from the upload.
df = pd.read_csv('COVID-19 Coronavirus.csv')

In [ ]:
#test data output
df.head()

,Country,Other names,ISO 3166-1 alpha-3 CODE,Population,Continent,TotalCases1,TotalDeaths1,Tot Cases//1M pop,Tot Deaths/1M pop,Death percentage
0,Afghanistan,Afghanistan,AFG,40462186,Asia,177827,7671,4395,190,4.313743
1,Albania,Albania,ALB,2872296,Europe,273870,3492,95349,1216,1.275058
2,Algeria,Algeria,DZA,45236699,Africa,265691,6874,5873,152,2.587216
3,Andorra,Andorra,AND,77481,Europe,40024,153,516565,1975,0.382271
4,Angola,Angola,AGO,34654212,Africa,99194,1900,2862,55,1.915438


In [ ]:
#testing use of Windows Functions (SQL) using Dataframe and Duckdb library
#This tells us what the mortality rate was compared to the number of confirmed cases.

request = duckdb.query("SELECT Country, round(sum(TotalDeaths1/TotalCases1)*100,2) AS 'DeathRatioPerCasesConfirmed' FROM df GROUP BY Country ORDER BY DeathRatioPerCasesConfirmed DESC")
print(request)

┌────────────────────────┬─────────────────────────────┐
│        Country         │ DeathRatioPerCasesConfirmed │
│        varchar         │           double            │
├────────────────────────┼─────────────────────────────┤
│ Yemen                  │                       18.15 │
│ Western Sahara         │                        10.0 │
│ Sudan                  │                        7.92 │
│ Peru                   │                        5.98 │
│ Mexico                 │                        5.71 │
│ Syria                  │                        5.64 │
│ Somalia                │                        5.11 │
│ Egypt                  │                        4.83 │
│ Afghanistan            │                        4.31 │
│ Bosnia and Herzegovina │                        4.18 │
│   ·                    │                          ·  │
│   ·                    │                          ·  │
│   ·                    │                          ·  │
│ Samoa                  │     

In [ ]:
#Invoking a Windows Function to structure the data further. This takes the row number, partitions by continent, then orders by mortality rate.

request = duckdb.query("SELECT Continent, Country, round(sum(TotalDeaths1/TotalCases1)*100,2) AS 'DeathRatioPerCasesConfirmed', ROW_NUMBER() OVER (PARTITION BY Continent) AS 'ID' FROM df GROUP BY Continent, Country ORDER BY Continent,DeathRatioPerCasesConfirmed DESC")
print(request)

┌───────────┬──────────────────┬─────────────────────────────┬───────┐
│ Continent │     Country      │ DeathRatioPerCasesConfirmed │  ID   │
│  varchar  │     varchar      │           double            │ int64 │
├───────────┼──────────────────┼─────────────────────────────┼───────┤
│ Africa    │ Western Sahara   │                        10.0 │    57 │
│ Africa    │ Sudan            │                        7.92 │    54 │
│ Africa    │ Somalia          │                        5.11 │    29 │
│ Africa    │ Egypt            │                        4.83 │     9 │
│ Africa    │ Liberia          │                        3.97 │    18 │
│ Africa    │ Niger            │                         3.5 │    24 │
│ Africa    │ Malawi           │                        3.07 │    19 │
│ Africa    │ Gambia           │                        3.04 │    43 │
│ Africa    │ Tunisia          │                        2.73 │    55 │
│ Africa    │ South Africa     │                        2.69 │    30 │
│   · 

In [ ]:
#Amalgamated view of mortality rate per continent.

request = duckdb.query("SELECT Continent, round(avg(TotalDeaths1/TotalCases1)*100,2) AS 'AverageDeathRatioPerCasesConfirmed' FROM df GROUP BY Continent ORDER BY AverageDeathRatioPerCasesConfirmed DESC")
request

┌─────────────────────────────────┬────────────────────────────────────┐
│            Continent            │ AverageDeathRatioPerCasesConfirmed │
│             varchar             │               double               │
├─────────────────────────────────┼────────────────────────────────────┤
│ Africa                          │                               1.99 │
│ Asia                            │                               1.56 │
│ Latin America and the Caribbean │                               1.54 │
│ Europe                          │                               0.99 │
│ Northern America                │                               0.71 │
│ Oceania                         │                               0.46 │
└─────────────────────────────────┴────────────────────────────────────┘

In [ ]:
#This creates an example CTE which can be used in the query, with variable inputs for Continent and population based on requirements. This then provides all countries in the given continent above the population threshold given.
continent_filter = 'Asia'
population_threshold = 100000000

request = duckdb.query(f"""
WITH ContinentCountries AS (
    SELECT
        Continent,
        Country,
        Population
    FROM df
    WHERE Continent = '{continent_filter}'
)
SELECT
    Country,
    Population
FROM ContinentCountries
WHERE Population > {population_threshold}
""")
request


┌─────────────┬────────────┐
│   Country   │ Population │
│   varchar   │   int64    │
├─────────────┼────────────┤
│ Bangladesh  │  167561502 │
│ China       │ 1439323776 │
│ India       │ 1403754381 │
│ Indonesia   │  278586508 │
│ Japan       │  125798669 │
│ Pakistan    │  228397520 │
│ Philippines │  112133868 │
└─────────────┴────────────┘

In [ ]:
#Wrapping input boxes around the required inputs to make this more user-friendly. Also putting the population into int() to mitigate random inputs.

continent_filter = input("Enter the continent: ")
population_threshold = int(input("Enter the minimum population threshold: "))

request = duckdb.query(f"""
WITH ContinentCountries AS (
    SELECT
        Continent,
        Country,
        Population
    FROM df
    WHERE Continent = '{continent_filter}'
)
SELECT
    Country,
    Population
FROM ContinentCountries
WHERE Population > {population_threshold}
""")
request

Enter the continent: Asia
Enter the minimum population threshold: 10000


┌─────────────┬────────────┐
│   Country   │ Population │
│   varchar   │   int64    │
├─────────────┼────────────┤
│ Afghanistan │   40462186 │
│ Armenia     │    2972939 │
│ Azerbaijan  │   10299156 │
│ Bahrain     │    1804995 │
│ Bangladesh  │  167561502 │
│ Bhutan      │     786480 │
│ Brunei      │     444812 │
│ Cambodia    │   17123941 │
│ China       │ 1439323776 │
│ Cyprus      │    1222745 │
│   ·         │       ·    │
│   ·         │       ·    │
│   ·         │       ·    │
│ Syria       │   18244381 │
│ Taiwan      │   23892241 │
│ Tajikistan  │    9912437 │
│ Thailand    │   70106601 │
│ Timor-Leste │    1362386 │
│ Turkey      │   85927644 │
│ UAE         │   10099567 │
│ Uzbekistan  │   34318156 │
│ Vietnam     │   98871712 │
│ Yemen       │   30975258 │
├─────────────┴────────────┤
│    49 rows (20 shown)    │
└──────────────────────────┘

In [ ]:
#In the case of error, 'try' included with except print statements in case of incorrect inputs.

try:
  continent_filter = input("Enter the continent: ")
  population_threshold = int(input("Enter the minimum population threshold: "))

  request = duckdb.query(f"""
  WITH ContinentCountries AS (
      SELECT
          Continent,
          Country,
          Population
      FROM df
      WHERE Continent = '{continent_filter}'
  )
  SELECT
      Country,
      Population
  FROM ContinentCountries
  WHERE Population > {population_threshold}
  """)
  request
except ValueError:
  print("Invalid input. Please enter an integer for the population threshold.")
except Exception as e:
  print(f"An error occurred: {e}")
